In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image
import plotly.offline as py

org_keywords = [
    'associate', 'committee', 'the other 98%', 'party', 'republicans',
    'physicians', 'USA', 'association', 'league', 'post', 'partnership',
    'think big', 'illinois', 'children at risk', 'union', 'leaders',
    'republican', 'coalition', '.org', 'services', '.com', 'fund', 'campaign',
    'amendments', 'americans', 'unite', 'entertainment', 'initiative',
    'austin', 'pundit', 'senate', 'agency', 'office', 'PAC', 'voice', 'news',
    'chamber', 'headlines', 'project', 'fellowship', 'politics', '.info',
    'liberty', 'report', 'university', 'press', 'institute', 'times', 'daily',
    'portal', 'revolution', 'world', 'department', 'network', 'school',
    'resistance', 'administration', 'council', 'taxpayers', 'foundation',
    'afscme', 'district', 'american', 'A Stronger Wisconsin', 'aclu',
    'new jersey', 'AFL-CIO', 'aarp', 'government', 'activist', 'actionaid',
    'information', 'wisconsin', 'america', 'alliance', 'list', 'house',
    'democrats', 'policy', 'constitution', 'tax', 'Ax The Bev Tax', 'oregon',
    'group', 'NJ', 'lives', 'NC', '.net', 'ohio', 'burger king', 'cnn',
    'catpac', 'californian', 'healthcare', 'hospital', 'action', 'citizen',
    'city', 'county', 'portland', 'clean water', 'club', 'college',
    'university', 'voice', 'common sense', 'fund', 'conservative', 'consumer',
    'deeds', 'dccc', 'street', 'service', 'social', 'democratic', 'doctors',
    'florida', 'society', 'espn', 'group', 'fellowship', 'conference',
    'democracy', 'floridians', 'forecast', 'project', 'advocates', 'partners',
    '.us', 'freedom', 'friends', 'future45', 'generation', 'georgia', 'bureau',
    'owners', 'lottery', 'polititions', 'senators', 'georgian', 'texas',
    'humanity'
]

data = pd.read_csv("cleaned_data.csv")


def source_type(x):
    for keyword in org_keywords:
        if keyword.lower() in x.lower() and 'facebook' not in x.lower():
            return 'Organization'
    return 'Individual'


df = data[["source", "renamed_veracity"]].copy()
df['source_type'] = df['source'].apply(lambda x: source_type(x))
top_5_sources = df.source.value_counts()[:5].index
fig = make_subplots(rows=2,
                    cols=4,
                    specs=[[{
                        "type": "pie",
                        "rowspan": 2,
                        "colspan": 2
                    }, None, {
                        "type": "pie"
                    }, {
                        "type": "pie"
                    }], [None, None, {
                        "type": "pie"
                    }, {
                        "type": "pie"
                    }]],
                    subplot_titles=("Sources of Data",
                                    "Veracity Distribution Across Top Sources",
                                    None, None, None),
                    vertical_spacing=0.01)

labels = [
    "Other Individuals", "Donald Trump", "Barack Obama", "Bloggers",
    "Facebook posts", "Hillary Clinton", "Organizations"
]
values = [11896, 816, 600, 502, 339, 297, 2300]
veracity_levels = [
    "true", "mostly-true", "half-true", "barely-true", "mostly-false", "false",
    "pants-fire"
]
fig_df = pd.DataFrame({'labels': labels, 'values': values})
fig.add_trace(
    go.Pie(
        labels=labels,
        values=values,
        customdata=fig_df,
        hovertemplate='Source: %{customdata[0][0]}' +
        '<br>Number of observations: %{customdata[0][1]}' +
        '<extra></extra>',  # this line is to prevent "trace 0" from appearing next to hover,
        texttemplate='%{percent:.1%f}',
        textposition='inside',
        textfont={
            'size': 20,
            'family': 'PT Sans Narrow'
        },
        hole=0.4,
        sort=False,
        direction='counterclockwise',
        legendgroup='',
        marker={
            'colors': [
                '#c6dbef', '#9ecae1', '#6baed6', '#4292c6', '#2171b5',
                '#084594', '#addd8e'
            ]
        },
        showlegend=False),
    row=1,
    col=1)
idx = 0

veracity_cats = [
    "True", "Mostly True", "Half True", "Barely True", "Mostly False", "False",
    "Pants on Fire!"
]

for row in [1, 2]:
    for col in [3, 4]:
        df_source = df[df.source == top_5_sources[idx]]
        values = []
        for level in veracity_levels:
            values.append(len(df_source[df_source.renamed_veracity == level]))

        custom_data = pd.DataFrame({
            'veracity_level': veracity_cats,
            'num_obs': values
        })
        fig.add_trace(
            go.Pie(
                labels=veracity_cats,
                values=values,
                customdata=custom_data,
                hovertemplate='Veracity: %{customdata[0][0]}' +
                '<br>Number of observations: %{customdata[0][1]}' +
                '<extra></extra>',  # this line is to prevent "trace 0" from appearing next to hover,
                texttemplate='%{percent:.1%f}',
                textposition='inside',
                textfont={
                    'size': 15,
                    'family': 'PT Sans Narrow'
                },
                hole=0.55,
                sort=False,
                direction='counterclockwise',
                legendgroup='',
                marker={
                    'colors': [
                        '#08519c', '#3182bd', '#6baed6', '#bdd7e7', '#fdbe85',
                        '#fd8d3c', '#e6550d'
                    ]
                },
                showlegend=True),
            row=row,
            col=col)
        idx = idx + 1

# add images for left pie chart
fig.add_layout_image(source=Image.open("trump.png"),
                     xref="paper",
                     yref="paper",
                     x=0.25,
                     y=1.01,
                     sizex=0.1,
                     sizey=0.1,
                     xanchor="right",
                     yanchor="bottom")
fig.add_layout_image(source=Image.open("obama.png"),
                     xref="paper",
                     yref="paper",
                     x=0.18,
                     y=0.98,
                     sizex=0.12,
                     sizey=0.12,
                     xanchor="right",
                     yanchor="bottom")
fig.add_layout_image(source=Image.open("blog.png"),
                     xref="paper",
                     yref="paper",
                     x=0.135,
                     y=0.94,
                     sizex=0.08,
                     sizey=0.08,
                     xanchor="right",
                     yanchor="bottom")
fig.add_layout_image(source=Image.open("fb.png"),
                     xref="paper",
                     yref="paper",
                     x=0.07,
                     y=0.88,
                     sizex=0.07,
                     sizey=0.07,
                     xanchor="right",
                     yanchor="bottom")
fig.add_layout_image(source=Image.open("hillary_clinton.png"),
                     xref="paper",
                     yref="paper",
                     x=0.08,
                     y=0.78,
                     sizex=0.13,
                     sizey=0.13,
                     xanchor="right",
                     yanchor="bottom")

# add images for the right pie chart
fig.add_layout_image(source=Image.open("trump.png"),
                     xref="paper",
                     yref="paper",
                     x=0.66,
                     y=0.72,
                     sizex=0.1,
                     sizey=0.1,
                     xanchor="right",
                     yanchor="bottom")
fig.add_layout_image(source=Image.open("obama.png"),
                     xref="paper",
                     yref="paper",
                     x=0.93,
                     y=0.70,
                     sizex=0.14,
                     sizey=0.14,
                     xanchor="right",
                     yanchor="bottom")
fig.add_layout_image(source=Image.open("blog.png"),
                     xref="paper",
                     yref="paper",
                     x=0.66,
                     y=0.2,
                     sizex=0.08,
                     sizey=0.08,
                     xanchor="right",
                     yanchor="bottom")
fig.add_layout_image(source=Image.open("fb.png"),
                     xref="paper",
                     yref="paper",
                     x=0.91,
                     y=0.21,
                     sizex=0.08,
                     sizey=0.08,
                     xanchor="right",
                     yanchor="bottom")

fig.update_layout(
    {
        'plot_bgcolor': 'rgba(8,29,88, 255)',
        'paper_bgcolor': 'rgba(8,29,88, 255)',
    },
    legend={
        "font": {
            "color": "white",
            "size": 25,
            "family": "PT Sans Narrow"
        }
    },
    margin=dict(l=20, r=20, t=200, b=20),
    height=800,
    width=1600)

for title in fig['layout']['annotations']:
    # to increase the font size of pie charts' titles
    title['font'] = {"size": 28}

# adjust the properties of title of left pie chart
fig['layout']['annotations'][0]['y'] = 1.18
fig['layout']['annotations'][0]['font']['color'] = "#ffffff"
fig['layout']['annotations'][0]['font']['family'] = "PT Sans Narrow"

# adjust the properties of title of right pie chart
fig['layout']['annotations'][1]['y'] = 1.18
fig['layout']['annotations'][1]['x'] = 0.78
fig['layout']['annotations'][1]['font']['color'] = "#ffffff"
fig['layout']['annotations'][1]['font']['family'] = "PT Sans Narrow"

py.plot(fig)
# fig.show()

'temp-plot.html'